## OpenAI에서 지원하는 Assistant API를 활용하여 PPT 파일을 생성해보자
- Assistant : 대화 및 파일생성, 코드 명령 수행, 데이터 검색 등 특정 task에 특화되어 학습된 자연어 기반 생성형 AI API
- OpenAI의 일반적인 Chat AI처럼 대화형 서비스도 제공하지만, Assistants는 특정 작업에 특화하여 지원하거나 자동화에 중점을 둠으로써 챗봇, 가상비서, 자동응답시스템 개발 등에 필요한 작업들에 포커스를 맞춰서 활용 가능
- OpenAI plarform의 API reference의 assistants 탭에서 확인 가능

### OpenAI API Key 설정

In [29]:
from openai import OpenAI
from getpass import getpass
import time

In [31]:
MY_API_KEY = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [32]:
client = OpenAI(api_key=MY_API_KEY)

### 어시스턴트, 스레드, 메세지 객체 생성
- 스레드(thread) : 프로세스 내에서 실행되는 가장 작은 실행 단위로 현재 실습에서는 어시스턴트와 사용자간의 채팅 세션(공간)을 하나의 스레드로 볼 수 있음
- 메세지(message) : 스레드 내의 통신 단위

In [ ]:
# <진행 프로세스>
# 1. 어시스턴트 세부사항 정의
# 2. 어시스턴트 객체 생성
# 3. 스레드 및 메세지 객체 생성
# 4. 스레드 실행(대화 시작 및 요구사항 요청)

#### 1) 어시스턴트 세부사항 정의

In [33]:
# 어시스턴트(모델)의 역할 정의(role에서 system과 유사한 기능)
assistant_instruction = """등산에 관련된 PowerPoint파일을 만들어야 해. 너는 등산 전문가이자 PowerPoint 제작 전문가야. 
전체적인 ppt의 글꼴은 알아보기 쉬운 분명한 한글 글꼴로 해야해.
페이지 별로 제목의 글씨 크기는 40point 내외, 내용은 20point정도로 설정해야해.
슬라이드는 3개를 만들어야 해.
"""

# 원하는 요청에 대한 자연어 질의 작성
prompt_user = """입문자 및 초보 등산가에게 강의하기 위한 프레젠테이션 자료를 만들어줘.
초급, 중급, 고급 수준별 적절한 등산 횟수, 산 높이, 기본적인 스트레칭 동작, 등산화 추천 등에 대한 설명을 넣어서 만들어줘.
내용은 구체적으로 작성해주면 좋겠어. 바로 작성해줘!
"""

#### 2) 어시스턴트 객체 생성

In [34]:
assistant = client.beta.assistants.create(name="My Assistant",                 # 이름은 원하는 데로 지정
                                           instructions=assistant_instruction, # 사전에 작성한 지침
                                           # Assistants API의 동작 방식 설정(code_interpreter, fucntion, file_search)
                                           tools=[{"type":"code_interpreter"}], 
                                           model="gpt-4o"
                                          )
# code_interpreter : 어시스턴트가 코드를 실행하여 계산, 데이터분석, 파일조작 등의 작업을 수행
# function : 특정 작업을 구현할 수 있는 사용자 정의 함수를 호출하여 작업 수행(사전에 사용자 정의 함수가 필요함)
# file_search : 사전 구성된 환경에서 파일을 검색하고 가져와 작업 수행

In [35]:
assistant

Assistant(id='asst_osJEA69VctQxO1bSh3fmSDmP', created_at=1754553476, description=None, instructions='등산에 관련된 PowerPoint파일을 만들어야 해. 너는 등산 전문가이자 PowerPoint 제작 전문가야. \n전체적인 ppt의 글꼴은 알아보기 쉬운 분명한 한글 글꼴로 해야해.\n페이지 별로 제목의 글씨 크기는 40point 내외, 내용은 20point정도로 설정해야해.\n슬라이드는 3개를 만들어야 해.\n', metadata={}, model='gpt-4o', name='My Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None)

#### 3) 스레드 및 메세지 객체 생성

In [36]:
# 경고 문구 미출력 코드
import warnings
warnings.filterwarnings(action='ignore')

In [37]:
# 스레드 객체 생성
 #  어시스턴트와 대화를 시작하면 스레드가 생성되며 해당 대화 중 교환되는 모든 메세지는 동일한 스레드에서 진행됨
thread = client.beta.threads.create()

# 메세지 객체 생성
 # 메세지는 단순히 사용자의 질의와 같은 텍스트 뿐만아니라 사용자와 어시스턴트 간에 교환되는 파일 등 모든 컨텐츠를 의미함
message = client.beta.threads.messages.create(thread_id=thread.id,   # 어떤 스레드에서 실행될지 스레드 id값 입력
                                              role="user",           # 메세지로 맨 처음 요청하는 것은 사용자이므로 user설정
                                              content=prompt_user    # 우리가 정의해둔 요청 입력
                                             )

#### 4) 스레드 실행(대화 시작 및 요구사항 요청)
- 채팅을 시작하여 위에 설정된 정보로 역할 설정 및 원하는 작업을 요청하는 단계

In [38]:
# 스레드 실행 (실행하면 LLM으로 사용자의 요청이 넘어감)
 # 스레드는 사용자가 요청을 입력했을 때 시작되고 모델의 응답이 끝나면 종료됨
run = client.beta.threads.runs.create(thread_id=thread.id,
                                      assistant_id=assistant.id
                                     )

- 생성형 모델이 사용자의 요청을 인지하고 결과물을 생성하기 위해서는 어느 정도의 시간이 필요함(복잡할수록 더욱 많은 시간이 필요함)

### 어시스턴트의 응답에 대한 진행상황 확인용 코드

In [39]:
while True : 
    # retieve : 특정 스레드의 실행 상태나 실행 후 결과를 검색하는데 사용되는 함수
    run_retrieve = client.beta.threads.runs.retrieve(thread_id=thread.id,    # 스레드 객체의 id
                                                     run_id=run.id           # 스레드 실행 객체의 id
                                                    )
    # 실행 완료인 경우
     # status는 in_progress, completed, failed로 출력 가능
    if run_retrieve.status == 'completed' :
        print("실행 완료!")
        break

    # 작업이 실행중이거나 실패한 경우
    else :
        print(run_retrieve.status)  # 해당 상태 출력
        time.sleep(3)               # 3초 텀을 두고 상태를 계속해서 출력

queued
in_progress
in_progress
in_progress
in_progress
in_progress
실행 완료!


In [40]:
# messages.list : 특정 스레드에서 오고간 메세지에 대한 정보를 출력하는 함수
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages

SyncCursorPage[Message](data=[Message(id='msg_9r4cYOSDJkVBZHiY7oN46M2G', assistant_id='asst_osJEA69VctQxO1bSh3fmSDmP', attachments=[Attachment(file_id='file-2TN88GGFeo6G2wkGfDsZmo', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=74, file_path=FilePath(file_id='file-2TN88GGFeo6G2wkGfDsZmo'), start_index=32, text='sandbox:/mnt/data/hiking_presentation.pptx', type='file_path')], value='프레젠테이션 자료가 준비되었습니다. [여기에서 다운로드](sandbox:/mnt/data/hiking_presentation.pptx)하실 수 있습니다. \n\n슬라이드에는 등산의 기본, 등산 난이도별 추천, 등산 전 준비 및 추천 용품에 대한 정보가 포함되어 있습니다. 각 슬라이드는 알아보기 쉬운 한글 글꼴로 작성되었으며, 추천된 글꼴 크기로 설정되었습니다.'), type='text')], created_at=1754553496, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_py7AHh22j8Rkx9I3bNO9yuhP', status=None, thread_id='thread_oYNZMfLhQxWtEgCZ6dM3M1hC'), Message(id='msg_D8MPmn16JSiDyySzHMQCk9Xd', assistant_id='asst_osJ

In [41]:
messages.data[-1]   # 맨 아래쪽 메세지가 우리가 요청한 메세지(위로 올라오면서 모델이 응답한 메세지들이 순차적으로 나오고 있음)

Message(id='msg_Mlh8meSrw543CoDQ06kkXe6S', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='입문자 및 초보 등산가에게 강의하기 위한 프레젠테이션 자료를 만들어줘.\n초급, 중급, 고급 수준별 적절한 등산 횟수, 산 높이, 기본적인 스트레칭 동작, 등산화 추천 등에 대한 설명을 넣어서 만들어줘.\n내용은 구체적으로 작성해주면 좋겠어. 바로 작성해줘!\n'), type='text')], created_at=1754553477, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_oYNZMfLhQxWtEgCZ6dM3M1hC')

In [42]:
# 메세지 객체를 통해 요청에 따른 응답 내용만 확인 (모델의 가장 마지막 응답 확인)
messages.data[0].content[0].text.value

'프레젠테이션 자료가 준비되었습니다. [여기에서 다운로드](sandbox:/mnt/data/hiking_presentation.pptx)하실 수 있습니다. \n\n슬라이드에는 등산의 기본, 등산 난이도별 추천, 등산 전 준비 및 추천 용품에 대한 정보가 포함되어 있습니다. 각 슬라이드는 알아보기 쉬운 한글 글꼴로 작성되었으며, 추천된 글꼴 크기로 설정되었습니다.'

In [43]:
# annotations : 메세지의 유형(텍스트, 이미지, 비디오, 각종파일 등)에 대한 정보가 담겨져 있음
messages.data[0].content[0].text.annotations

[FilePathAnnotation(end_index=74, file_path=FilePath(file_id='file-2TN88GGFeo6G2wkGfDsZmo'), start_index=32, text='sandbox:/mnt/data/hiking_presentation.pptx', type='file_path')]

#### annotations가 비어있는 경우의 원인
- 생성 모델이 결과물을 전부 생성하기 전에 코드를 실행했을 경우
- 필요한 데이터가  메세지에 포함되지 않았을 경우(지시가 애매했거나 파일이 제대로 첨부되지 않았을 때)
- API 호출에 문제가 있거나 잘못된 trained_id를 사용하여 올바른 데이터를 가져오지 못한 경우
- 지정한 생성형 AI의 성능으로 완료할 수 없는 어려운 요청일 경우

### 완료된 작업(파일 생성)에 대한 정보 추출 및 실제 파일로 내보내기

In [44]:
# 메세지를 통해 받은 생성한 파일 정보에 대한 file_id값을 저장
file_id_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_id_path

'file-2TN88GGFeo6G2wkGfDsZmo'

In [45]:
# file_id 값을 통해 생성된 파일의 내용들을 검색해 가져오는 함수
file_contents = client.files.with_raw_response.retrieve_content(file_id_path)

In [46]:
with open("PPT_001.pptx", "wb") as f :
    # 메세지에서 가져온 파일의 정보를 바탕으로 실제 생성할 파일에 컨텐츠 입히기(쓰기)
    f.write(file_contents.content)

### 정리
- 간단한 요청으로 진행했기 때문에 결과물의 퀄리티는 높지 않지만 Assistant API를 활용하면 일반 대화형 API로 하기 힘든 파일 제작 등 특정 task에 특화된 응답과 결과를 얻을 수 있음